# Understanding Greek Government Procedures


*   Identify the types of procedures and the kinds of documents, policies, or laws involved (e.g., tax regulations, immigration rules, business licensing).
*   Access publicly available government datasets, portals (like gov.gr), or APIs for structured information.




In [1]:
!pip install llama-index llama-index-readers-file llama-index-llms-openai llama-index-embeddings-huggingface llama-index-llms-litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, download_loader, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SemanticSplitterNodeParser, SimpleNodeParser
from llama_index.llms.openai import OpenAI as LLamaIndexOpenAI
from llama_index.llms.litellm import LiteLLM

In [3]:
!pip install openai pymupdf tqdm chromadb spacy sentence-transformers
!python -m spacy download  el_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 5.7 MB/s eta 0:00:00

# **1. Data Ingestion**

In [4]:
import requests
import re
import os
import shutil
import uuid
import fitz
import urllib.request
from tqdm.autonotebook import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import logging
from typing import Set, List, Optional
import json


def download_pdf(url, output_path):
    os.makedirs(output_path, exist_ok=True)
    local_pdf = f"{output_path}/{uuid.uuid4().hex}.pdf"
    if url == output_path:
         return
    try:
        urllib.request.urlretrieve(url, local_pdf)
    except ValueError:
        shutil.copy(url, local_pdf)
    return local_pdf

def preprocess(text):
    text = text.replace("-\n", "")  # no word breaks
    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\.+", ".", text)
    return text

def pdf2text(path, start_page=0, end_page=-1):
    print("Parsing PDF")
    doc = fitz.open(path)
    total_pages = doc.page_count
    print(f"PDF contains {total_pages} pages")
    if end_page <= 0:
        end_page = total_pages

    text_list = []
    for i in tqdm(range(start_page, end_page), desc=f"Converting PDF to text. Pages: {start_page}-{end_page}"):
        text = doc.load_page(i).get_text("text")
        text = preprocess(text)
        text_list.append(text)
    doc.close()
    return " ".join(text_list)

class WebCrawler:
    def __init__(
        self,
        base_url: str,
        max_pages: int = 100,
        delay: float = 1.0,
        respect_robots: bool = True,
        save_path: Optional[str] = None
    ):
        """
        Initialize the web crawler

        Args:
            base_url: The starting URL to crawl
            max_pages: Maximum number of pages to crawl
            delay: Delay between requests in seconds
            respect_robots: Whether to respect robots.txt
            save_path: Path to save the crawled data (optional)
        """
        self.base_url = base_url
        self.max_pages = max_pages
        self.delay = delay
        self.respect_robots = respect_robots
        self.save_path = save_path
        self.visited_urls: Set[str] = set()
        self.data = []

        # Setup logging
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    def is_valid_url(self, url: str) -> bool:
        """Check if URL belongs to the same domain and is valid"""
        try:
            parsed_base = urlparse(self.base_url)
            parsed_url = urlparse(url)
            return (
                parsed_url.netloc == parsed_base.netloc
                and parsed_url.scheme in ['http', 'https']
            )
        except Exception:
            return False

    def extract_text_and_links(self, soup: BeautifulSoup, url: str) -> tuple[str, List[str]]:
        """Extract text content and links from a page"""
        # Remove script and style elements
        for script in soup(['script', 'style', 'header', 'footer', 'nav']):
            script.decompose()

        # Get text content
        text = ' '.join(soup.stripped_strings)

        # Extract links
        links = []
        for link in soup.find_all('a', href=True):
            absolute_url = urljoin(url, link['href'])
            if self.is_valid_url(absolute_url):
                links.append(absolute_url)

        return text, links

    def crawl(self):
        """Start the crawling process"""
        urls_to_visit = [self.base_url]

        while urls_to_visit and len(self.visited_urls) < self.max_pages:
            url = urls_to_visit.pop(0)

            if url in self.visited_urls:
                continue

            try:
                self.logger.info(f"Crawling: {url}")

                # Respect crawl delay
                time.sleep(self.delay)

                # Fetch and parse the page
                response = requests.get(url, timeout=10)
                response.raise_for_status()

                soup = BeautifulSoup(response.text, 'html.parser')
                text, new_links = self.extract_text_and_links(soup, url)

                # Store the data
                page_data = {
                    'url': url,
                    'title': soup.title.string if soup.title else '',
                    'text': text,
                    'timestamp': time.time()
                }
                self.data.append(page_data)


                if page_data['url'][len(page_data['url'])-4:] == ".pdf":
                    local_pdf_greek = download_pdf(page_data['url'], "data")
                    text_greek = pdf2text(local_pdf_greek, start_page=0, end_page=-1)
                    page_data['text'] = text_greek

                # Save progress if save_path is specified
                if self.save_path:
                    self.save_data()

                # Add new links to visit
                urls_to_visit.extend([
                    link for link in new_links
                    if link not in self.visited_urls
                ])

                self.visited_urls.add(url)

            except Exception as e:
                self.logger.error(f"Error crawling {url}: {str(e)}")
                continue

        self.logger.info(f"Crawling completed. Visited {len(self.visited_urls)} pages.")


    def save_data(self):
        """Save crawled data to a JSON file"""

        if self.save_path:
            with open(self.save_path, 'w', encoding='utf-8') as f:
                json.dump(self.data, f, ensure_ascii=False, indent=2)

# Example usage
if __name__ == "__main__":
    crawler = WebCrawler(
        base_url="https://mitos.gov.gr/index.php/Αρχική_σελίδα",
        max_pages=200,
        delay=1.0,
        save_path="crawled_data.json"
    )
    crawler.crawl()

Parsing PDF
PDF contains 1 pages


Converting PDF to text. Pages: 0-1:   0%|          | 0/1 [00:00<?, ?it/s]

## 2. **Semantic chunking**

In [5]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings



In [6]:
with open("crawled_data.json", "r") as f:
    crawled_data = json.load(f)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

dataset = []

for page in crawled_data:
    text = page["text"]
    url = page["url"]
    title = page["title"]

    dataset.append(text)

print(f"Inserted {len(dataset)} documents into the collection.")

# Create a temporary directory
temp_dir = "temp_texts"
os.makedirs(temp_dir, exist_ok=True)

# Write each text to a file
for idx, text in enumerate(dataset):
    file_path = os.path.join(temp_dir, f"doc_{idx + 1}.txt")
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(text)

# Use SimpleDirectoryReader to read the directory
reader = SimpleDirectoryReader(temp_dir)
documents = reader.load_data()

# Cleanup (Optional): Remove the temporary directory and files
shutil.rmtree(temp_dir)

nodes = splitter.get_nodes_from_documents(documents)

print(nodes[2])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Inserted 200 documents into the collection.
Node ID: b8f2774e-4097-47e9-a1d2-ebcce55e7d43
Text: Κατηγορία:Επιχειρηματική δραστηριότητα - Εθνικό Μητρώο
Διοικητικών Διαδικασιών Εθνικό Μητρώο Διοικητικών Διαδικασιών Σύνδεση
ΕL ΕN Βοήθεια Κατηγορία:Επιχειρηματική δραστηριότητα Από Εθνικό Μητρώο
Διοικητικών Διαδικασιών Μετάβαση σε: πλοήγηση , αναζήτηση
Υποκατηγορίες Αυτή η κατηγορία έχει τις ακόλουθες 12 υποκατηγορίες,
από 12 συνολικά. Α ► Αδει...


## 3. **Query the LLM**

In [7]:
from llama_index.core.prompts import (
    SelectorPromptTemplate,
    PromptType,
    PromptTemplate,
    MessageRole,
    ChatPromptTemplate,
    ChatMessage,
)

index = VectorStoreIndex(nodes, embed_model=embed_model)

QA_PROMPT = """BEGININPUT
{context_str}

ENDINPUT
BEGININSTRUCTION
{query_str}
ENDINSTRUCTION"""

QA_SYSTEM_PROMPT = """Είσαι ένας ειλικρινής και αμερόληπτος βοηθός που πάντα απαντάει με ακρίβεια σε αυτά που του ζητούνται.
Σου δίνεται ένα έγγραφο το οποίο βρίσκεται μεταξύ του BEGININPUT και του ENDINPUT.
Επίσης, σου δίνονται μεταδεδομένα για το συγκεκριμένο έγγραφο μεταξύ του BEGINCONTEXT και του ENDCONTEXT.
Το βασικό κείμενο του εγγράφου βρίσκεται μεταξύ του ENDCONTEXT και του ENDINPUT.
Απάντα στις οδηγίες του χρήστη που βρίσκονται μεταξύ του BEGININSTRUCTION και του ENDINSTRUCTION χρησιμοποιώντας μόνο το βασικό κείμενο
και τα μεταδεδομένα του εγγράφου που σου δίνονται παρακάτω. Αν οι οδηγίες που σου ζητάει ο χρήστης δεν μπορούν να απαντηθούν με το βασικό
κείμενο ή τα μεταδεδομένα του εγγράφου, ενημέρωσε τον χρήστη ότι δεν ξέρεις την σωστή απάντηση."""

REFINE_PROMPT = """Η αρχική ερώτηση είναι η εξής: {query_str}
Έχουμε δώσει την παρακάτω απάντηση: {existing_answer}
Έχουμε την ευκαιρία να βελτιώσουμε την προηγούμενη απάντηση (μόνο αν χρειάζεται) με τις παρακάτω νέες πληροφορίες.
------------
{context_str}
------------
Με βάση τις νέες πληροφορίες, βελτίωσε την απάντησή σου για να απαντά καλύτερα την ερώτηση.
Αν οι πληροφορίες δεν είναι χρήσιμες, απλά επανέλαβε την προηγούμενη απάντηση.
Βελτιωμένη απάντηση: """

REFINE_CHAT_PROMPT = """Είσαι ένα έμπιστο σύστημα που απαντάει ερωτήσεις χρηστών. Λειτουργείς με τους εξής δύο τρόπους όταν βελτιώνεις υπάρχουσες απαντήσεις:
1. **Ξαναγράφεις** την απάντηση με βάση νέες πληροφορίες που σου παρέχονται
2. **Επαναλαμβάνεις** την προηγούμενη απάντηση αν δεν είναι χρήσιμες οι νέες πληροφορίες

Ποτέ δεν αναφέρεις την αρχική απάντηση ή το συγκείμενο απευθείας στην απάντησή σου.
Αν υπάρχει αμφιβολία για το τι πρέπει να απαντήσεις, απλά επανέλαβε την αρχική απάντηση.
Νέες πληροφορίες:
------------
{context_str}
------------
Ερώτηση: {query_str}
Αρχική απάντηση: {existing_answer}
Βελτιωμένη απάντηση: """


default_template = PromptTemplate(
    metadata={"prompt_type": PromptType.QUESTION_ANSWER},
    # template_vars=["context_str", "query_str"],
    template=QA_PROMPT,
)
chat_template = ChatPromptTemplate(
    metadata={"prompt_type": PromptType.CUSTOM},
    # template_vars=["context_str", "query_str"],
    message_templates=[
        ChatMessage(role=MessageRole.SYSTEM, content=QA_SYSTEM_PROMPT),
        ChatMessage(role=MessageRole.USER, content=QA_PROMPT),
    ],
)
text_qa_prompt = SelectorPromptTemplate(
    default_template=default_template, conditionals=[(lambda llm: True, chat_template)]
)

default_refine_template = PromptTemplate(
    metadata={"prompt_type": PromptType.REFINE},
    # template_vars=["query_str", "existing_answer", "context_str"],
    template=REFINE_PROMPT,
)
chat_refine_template = ChatPromptTemplate(
    metadata={"prompt_type": PromptType.CUSTOM},
    # template_vars=["context_str", "query_str", "existing_answer"],
    message_templates=[ChatMessage(role=MessageRole.USER, content=REFINE_CHAT_PROMPT)],
)
text_refine_prompt = SelectorPromptTemplate(
    default_template=default_refine_template,
    conditionals=[(lambda llm: True, chat_refine_template)],
)

meltemi = LiteLLM(
    "hosted_vllm/meltemi-vllm",
    api_base="http://ec2-3-19-37-251.us-east-2.compute.amazonaws.com:4000/",
    api_key="sk-RYF0g_hDDIa2TLiHFboZ1Q",
)
query_engine = index.as_query_engine(
    llm=meltemi, text_qa_prompt=text_qa_prompt, text_refine_prompt=text_refine_prompt
)

query_engine.query("Τι είναι το Μητρώο Διαδικασιών Δημόσιας Διοίκησης Μίτος και ποιος είναι ο στόχος του;").response

'Το Μητρώο Διαδικασιών Δημόσιας Διοίκησης Μίτος είναι ένα συνεργατικό εργαλείο της δημόσιας διοίκησης που στοχεύει στην τυποποίηση των διοικητικών διαδικασιών, στην ενίσχυση της διαφάνειας και της ασφάλειας δικαίου, και στην έγκυρη και αξιόπιστη πληροφόρηση σε φυσικά και νομικά πρόσωπα, νομικές οντότητες, δημοσίους υπαλλήλους και λειτουργούς σχετικά με τη λειτουργία της δημόσιας διοίκησης.'

In [8]:
query_engine.query("Πώς μπορώ να βρω πληροφορίες για μια συγκεκριμένη διαδικασία μέσω της πλατφόρμας Μίτος;").response

'Βεβαίως, θα χαρώ πολύ να βοηθήσω!\n\nΓια να βρείτε πληροφορίες για μια συγκεκριμένη διαδικασία μέσω της πλατφόρμας Μίτος, μπορείτε να χρησιμοποιήσετε τη λειτουργία αναζήτησης που βρίσκεται στην πάνω δεξιά γωνία της σελίδας. Μπορείτε επίσης να πλοηγηθείτε στις διαφορετικές κατηγορίες υπηρεσιών που παρέχει η πλατφόρμα, όπως Γεωργία και Γεωργία, Εκπαίδευση, Επιχειρηματικές Δραστηριότητες, Εργασία και Ασφάλιση, Οικογένεια, Ακίνητη Περιουσία και Φορολογία, Ιθαγένεια, Πολιτισμός, Αθλητισμός και Τουρισμός, και Στρατιωτική Υπηρεσία.\n\nΕπιπλέον, μπορείτε να αναζητήσετε διαδικασίες με βάση τον τύπο τους, όπως Άδειες, Ανανεώσεις, Αποζημιώσεις, Βεβαιώσεις, Δηλώσεις, Εγγραφές, Έλεγχοι, Εσωτερικές διαδικασίες δημοσίου, Καταγγελίες, Πιστοποιητικά, Συντάξεις και Σύντομη παρουσίαση του.\n\nΕλπίζω να βοήθησα! Αν έχετε περαιτέρω ερωτήσεις ή χρειάζεστε περισσότερες πληροφορίες, μη διστάσετε να ρωτήσετε.'

In [9]:
query_engine.query("Ποιες είναι οι κύριες κατηγορίες διαδικασιών που περιλαμβάνονται στη βάση δεδομένων του Μίτος;").response

'Οι κύριες κατηγορίες διαδικασιών που περιλαμβάνονται στη βάση δεδομένων του Μίτος είναι:\n\n1. Άδειες\n2. Αιτήσεις\n3. Αναγγελίες\n4. Αναθέσεις\n5. Ανακλήσεις\n6. Ανανεώσεις\n7. Αξιολογήσεις\n8. Απαγορεύσεις\n9. Απαλλοτριώσεις\n10. Απογραφές\n11. Αποζημιώσεις\n12. Απονομές\n13. Αποσπάσεις\n14. Αποφάσεις\n15. Αρχαιρεσίες\n16. Βεβαιώσεις\n17. Βραβεύσεις\n18. Γνωμοδοτήσεις\n19. Γνωστοποιήσεις\n20. Δηλώσεις\n21. Δημοπρασίες\n22. Δημόσιες Συμβάσεις\n23. Διαγωνισμοί\n24. Διαπιστωτικές Πράξεις\n25. Διαχείριση\n26. Διεκπεραιώσεις\n27. Διορισμοί\n28. Δωρεές\n29. Εγγραφές\n30. Εγκρίσεις\n31. Εισηγήσεις\n32. Εισφορές\n33. Εκθέσεις\n34. Εκμεταλλεύσεις\n35. Εκπαιδεύσεις/Επιμορφώσεις\n36. Εκποιήσεις\n37. Έλεγχοι\n38. Ενημερτότητες\n39. Ενστάσεις/προσφυγές\n40. Εξετάσεις\n41. Εξουσιοδοτήσεις\n42. Εξοφλήσεις\n43. Επανεκδόσεις\n44. Επιδόματα πολιτών\n45. Επικυρώσεις\n46. Επιχορηγήσεις\n47. Εσωτερικές διαδικασίες\n48. Θεωρήσεις\n49. Ιδρύσεις\n50. Ισχυρισμοί\n51. Καταβολές\n52. Καταγγελίες επιχειρήσεων\

In [11]:
query_engine.query("Τι σχέση έχει η πλατφόρμα Μίτος με το μίτο της Αριάδνης;").response

'Η πλατφόρμα "Μίτος" πήρε το όνομά της από το μύθο του μίτου της Αριάδνης, ο οποίος συμβολίζει την παροχή καθοδήγησης και βοήθειας σε δύσκολες καταστάσεις. Όπως ο μίτος της Αριάδνης βοήθησε τον Θησέα να βρει το δρόμο του πίσω από το λαβύρινθο, έτσι και η πλατφόρμα "Μίτος" βοηθά τους χρήστες να πλοηγηθούν μέσα από τις διοικητικές διαδικασίες, παρέχοντας σαφείς και περιεκτικές πληροφορίες.\n\nΗ πλατφόρμα "Μίτος" αναπτύχθηκε από το Υπουργείο Εσωτερικών της Ελλάδας για να παρέχει μια ολοκληρωμένη επισκόπηση των διοικητικών διαδικασιών που απαιτούνται για διάφορες υπηρεσίες. Η πλατφόρμα σχεδιάστηκε για να βοηθήσει τους πολίτες και τους δημόσιους υπαλλήλους να πλοηγηθούν αποτελεματικά μέσα από τις διοικητικές διαδικασίες, μειώνοντας την γραφειοκρατία και απλοποιώντας τις διαδικασίες.\n\nΣυνοψίζοντας, η πλατφόρμα "Μίτος" πήρε το όνομά της από το μύθο του μίτου της Αριάδνης, συμβολίζοντας την παροχή καθοδήγησης και βοήθειας σε δύσκολες καταστάσεις, όπως ακριβώς και ο μύθος. Η πλατφόρμα σχεδιάσ

In [13]:
query_engine.query("Πόσο μακρύς είναι ο Μίτος;").response

'Το μήκος του Μίτου δεν αναφέρεται στο κείμενο.'